<a href="https://colab.research.google.com/github/NewTAs/GoogLeNet/blob/main/GoogLeNet_Step7_DriveVer_Epoch10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step1 Code
import os
import re
import glob
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf

from google.colab import drive
drive.mount('googlenet')

# New Cord
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

#np.random.seed(4) # 랜덤시드 고정시키기

Mounted at googlenet


In [2]:
numberOfClass = 30
pictureWeightHeightSize = 227

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/content/googlenet/MyDrive/GoogLeNet_Data/TrainData_Step7/',
        target_size=(pictureWeightHeightSize, pictureWeightHeightSize),
        batch_size=numberOfClass,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        '/content/googlenet/MyDrive/GoogLeNet_Data/TestData_Step7/',
        target_size=(pictureWeightHeightSize, pictureWeightHeightSize),    
        batch_size=numberOfClass,
        class_mode='categorical')

Found 300 images belonging to 30 classes.
Found 60 images belonging to 30 classes.


In [3]:
input_layer = tf.keras.layers.Input(shape=(pictureWeightHeightSize, pictureWeightHeightSize, 3))

In [4]:
stem_layer = tf.keras.layers.Conv2D(32, (3, 3), strides=(1, 1), padding='SAME', activation='relu')(input_layer)
# Max Pooling Here on Original Inception (3x3, stride=2)
stem_layer = tf.keras.layers.Conv2D(64, (3, 3), strides=(1, 1), padding='SAME', activation='relu')(stem_layer)
# Max Pooling Here on Original Inception (3x3, stride=2)

In [5]:
inception_block3a_1x1 = tf.keras.layers.Conv2D(32, (1, 1), strides=(1, 1), padding='SAME', activation='relu')(stem_layer)

inception_block3a_3x3 = tf.keras.layers.Conv2D(48, (1, 1), padding='SAME', activation='relu')(stem_layer)
inception_block3a_3x3 = tf.keras.layers.Conv2D(64, (3, 3), padding='SAME', activation='relu')(inception_block3a_3x3)

inception_block3a_5x5 = tf.keras.layers.Conv2D(8, (1, 1), padding='SAME', activation='relu')(stem_layer)
inception_block3a_5x5 = tf.keras.layers.Conv2D(16, (5, 5), padding='SAME', activation='relu')(inception_block3a_5x5)

inception_block3a_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(stem_layer)
inception_block3a_pool = tf.keras.layers.Conv2D(16, (1, 1), padding='SAME', activation='relu')(inception_block3a_pool)

inception_block3a = tf.keras.layers.Concatenate()([inception_block3a_1x1, inception_block3a_3x3,
                                                   inception_block3a_5x5, inception_block3a_pool]) # 32x32x128

In [6]:
# inception_block3b Here on Original Inception
max_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(2, 2), padding='SAME')(inception_block3a) # 16x16x128

In [7]:
inception_block4a_1x1 = tf.keras.layers.Conv2D(64, (1, 1), padding='SAME', activation='relu')(max_pool)

inception_block4a_3x3 = tf.keras.layers.Conv2D(64, (1, 1), padding='SAME', activation='relu')(max_pool)
inception_block4a_3x3 = tf.keras.layers.Conv2D(96, (3, 3), padding='SAME', activation='relu')(inception_block4a_3x3)

inception_block4a_5x5 = tf.keras.layers.Conv2D(16, (1, 1), padding='SAME', activation='relu')(max_pool)
inception_block4a_5x5 = tf.keras.layers.Conv2D(48, (5, 5), padding='SAME', activation='relu')(inception_block4a_5x5)

inception_block4a_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(max_pool)
inception_block4a_pool = tf.keras.layers.Conv2D(32, (1, 1), padding='SAME', activation='relu')(inception_block4a_pool)

inception_block4a = tf.keras.layers.Concatenate()([inception_block4a_1x1, inception_block4a_3x3,
                                                   inception_block4a_5x5, inception_block4a_pool]) # 16x16x240

# inception_block4b Here on Original Inception
# inception_block4c Here on Original Inception
# inception_block4d Here on Original Inception
# inception_block4e Here on Original Inception

In [8]:
max_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(2, 2), padding='SAME')(inception_block4a) # 8x8x240

In [9]:
inception_block5a_1x1 = tf.keras.layers.Conv2D(128, (1, 1), padding='SAME', activation='relu')(max_pool)

inception_block5a_3x3 = tf.keras.layers.Conv2D(80, (1, 1), padding='SAME', activation='relu')(max_pool)
inception_block5a_3x3 = tf.keras.layers.Conv2D(120, (3, 3), padding='SAME', activation='relu')(inception_block5a_3x3)

inception_block5a_5x5 = tf.keras.layers.Conv2D(16, (1, 1), padding='SAME', activation='relu')(max_pool)
inception_block5a_5x5 = tf.keras.layers.Conv2D(64, (5, 5), padding='SAME', activation='relu')(inception_block5a_5x5)

inception_block5a_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(max_pool)
inception_block5a_pool = tf.keras.layers.Conv2D(64, (1, 1), padding='SAME', activation='relu')(inception_block5a_pool)

inception_block5a = tf.keras.layers.Concatenate()([inception_block5a_1x1, inception_block5a_3x3,
                                                   inception_block5a_5x5, inception_block5a_pool]) # 8x8x416
# inception_block5b Here on Original Inception

In [10]:
avg_pool = tf.keras.layers.GlobalAveragePooling2D()(inception_block5a) # 416
dropout = tf.keras.layers.Dropout(0.4)(avg_pool)
out_layer = tf.keras.layers.Dense(numberOfClass, activation='softmax')(dropout)

In [11]:
model = tf.keras.models.Model(input_layer, out_layer)

#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

#model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 227, 227, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 227, 227, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 227, 227, 64) 18496       conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 227, 227, 48) 3120        conv2d_1[0][0]                   
______________________________________________________________________________________________

In [12]:
model.fit(train_generator, epochs=10, validation_data=test_generator)

Epoch 1/10
10/10 [==============================] - 388s 38s/step - loss: 3.4087 - accuracy: 0.0200 - val_loss: 3.4026 - val_accuracy: 0.0333
Epoch 2/10
10/10 [==============================] - 361s 36s/step - loss: 3.3889 - accuracy: 0.0267 - val_loss: 3.4100 - val_accuracy: 0.0333
Epoch 3/10
10/10 [==============================] - 353s 36s/step - loss: 3.3544 - accuracy: 0.0267 - val_loss: 3.4027 - val_accuracy: 0.0333
Epoch 4/10
10/10 [==============================] - 345s 35s/step - loss: 3.2684 - accuracy: 0.0500 - val_loss: 3.4227 - val_accuracy: 0.0333
Epoch 5/10
10/10 [==============================] - 353s 35s/step - loss: 3.1811 - accuracy: 0.0733 - val_loss: 3.5249 - val_accuracy: 0.0667
Epoch 6/10
10/10 [==============================] - 347s 35s/step - loss: 3.0200 - accuracy: 0.0767 - val_loss: 3.6249 - val_accuracy: 0.0333
Epoch 7/10
10/10 [==============================] - 344s 35s/step - loss: 2.9083 - accuracy: 0.0800 - val_loss: 3.7179 - val_accuracy: 0.0333
Epoch 

In [13]:
print("-- Evaluate --")
scores = model.evaluate(test_generator)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
2/2 [==============================] - 13s 7s/step - loss: 4.9318 - accuracy: 0.0667
accuracy: 6.67%


In [14]:
print("-- Predict --")
output = model.predict(test_generator)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

-- Predict --
{'0': 0, '1': 1, '10': 2, '11': 3, '12': 4, '13': 5, '14': 6, '15': 7, '16': 8, '17': 9, '18': 10, '19': 11, '2': 12, '20': 13, '21': 14, '22': 15, '23': 16, '24': 17, '25': 18, '26': 19, '27': 20, '28': 21, '29': 22, '3': 23, '4': 24, '5': 25, '6': 26, '7': 27, '8': 28, '9': 29}
[[0.007 0.035 0.000 ... 0.002 0.020 0.031]
 [0.002 0.000 0.031 ... 0.020 0.000 0.000]
 [0.065 0.039 0.026 ... 0.050 0.036 0.022]
 ...
 [0.001 0.012 0.000 ... 0.000 0.003 0.008]
 [0.039 0.067 0.008 ... 0.023 0.060 0.052]
 [0.014 0.001 0.073 ... 0.059 0.004 0.001]]


In [17]:
for i in range(0, 30):
  print(output[i])
  print("\n")

[0.007 0.035 0.000 0.001 0.001 0.036 0.122 0.002 0.128 0.032 0.002 0.095
 0.006 0.070 0.011 0.114 0.000 0.000 0.007 0.057 0.000 0.003 0.026 0.000
 0.107 0.083 0.000 0.002 0.020 0.031]


[0.002 0.000 0.031 0.035 0.041 0.000 0.000 0.003 0.000 0.001 0.030 0.000
 0.002 0.000 0.001 0.000 0.659 0.011 0.020 0.000 0.069 0.004 0.002 0.049
 0.000 0.000 0.019 0.020 0.000 0.000]


[0.065 0.039 0.026 0.034 0.034 0.061 0.001 0.039 0.008 0.045 0.036 0.017
 0.059 0.048 0.048 0.000 0.010 0.025 0.066 0.058 0.023 0.036 0.076 0.023
 0.000 0.001 0.013 0.050 0.036 0.022]


[0.008 0.036 0.000 0.001 0.001 0.038 0.154 0.002 0.131 0.025 0.002 0.093
 0.007 0.070 0.011 0.100 0.000 0.001 0.008 0.048 0.000 0.003 0.022 0.000
 0.098 0.091 0.000 0.002 0.018 0.029]


[0.016 0.042 0.001 0.001 0.001 0.091 0.210 0.003 0.085 0.024 0.001 0.103
 0.008 0.123 0.009 0.044 0.000 0.000 0.008 0.049 0.000 0.003 0.022 0.000
 0.060 0.068 0.000 0.003 0.008 0.017]


[0.008 0.050 0.003 0.007 0.005 0.023 0.022 0.009 0.122 0.046 0.014 0.0